In [2]:
import cv2
import Tkinter as tk
import tkFileDialog
import os
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [3]:
def load_images(folder):
        image_list=[]
        for filename in os.listdir(folder):
            img=cv2.imread(os.path.join(folder,filename))
            if img is not None:
                image_list.append(img)
        return image_list

In [4]:
tk.Tk().withdraw()
path=tkFileDialog.askdirectory()
print path


/home/edward/project /Untitled Folder


In [5]:
image=load_images(path)

In [6]:
data=pd.read_csv("/home/edward/Skin Cancer Data/ISIC-2017_Training_Part3_GroundTruth.csv")

In [7]:
data['melanoma'].head()

0    0
1    0
2    1
3    0
4    1
Name: melanoma, dtype: int64

In [8]:
def resize_images(images):
    resize_image=[]
    for image in images:
        imag=cv2.resize(image,(64,64))
        resize_image.append(imag)
    return resize_image

In [9]:
image=resize_images(image)
image=np.array(image)


In [10]:
image1=image[0:50]

In [11]:
image1.shape

(50, 64, 64, 3)

In [12]:
image=image1

In [31]:
image.shape

(50, 64, 64, 3)

In [35]:
learning_rate=0.001
training_epoch=15
batch_size=50
display_step=1

n_hidden_1=256
n_hidden_2=256
#n_hidden_3=256
#n_hidden_4=256
n_input=50*64*64*3
n_classes=2

x=tf.placeholder("float",[None,n_input])
y=tf.placeholder("float",[None,n_classes])

In [36]:
def multiplayer_perceptron(x,weights,biases):
    # first layer
    #with tf.device('/gpu:0'):
    layer_1=tf.add(tf.matmul(x,weights['h1']),biases['b1'])
    layer_1=tf.nn.sigmoid(layer_1)
    # second layer
    #with tf.device('/gpu:1'):
    layer_2=tf.add(tf.matmul(layer_1,weights['h2']),biases['b2'])
    layer_2=tf.nn.sigmoid(layer_2)
    # third layer
    """
    layer_3=tf.add(tf.matmul(layer_2,weights['h3']),biases['b3'])
    layer_3=tf.nn.relu(layer_3)
    # forth layer
    layer_4=tf.add(tf.matmul(layer_3,weights['h4']),biases['b4'])
    layer_4=tf.nn.relu(layer_4)
    # output layer

    """
    out_layer=tf.matmul(layer_2,weights['out'])+biases['out']
    return out_layer

In [37]:
# store layer weights and biases
weights={
    'h1':tf.Variable(tf.random_normal([n_input,n_hidden_1])),
    'h2':tf.Variable(tf.random_normal([n_hidden_1,n_hidden_2])),
    #'h3':tf.Variable(tf.random_normal([n_hidden_2,n_hidden_3])),
    #'h4':tf.Variable(tf.random_normal([n_hidden_3,n_hidden_4])),
    'out':tf.Variable(tf.random_normal([n_hidden_1,n_classes]))
}

biases={
    'b1':tf.Variable(tf.random_normal([n_hidden_1])),
    'b2':tf.Variable(tf.random_normal([n_hidden_2])),
    #'b3':tf.Variable(tf.random_normal([n_hidden_3])),
    #'b4':tf.Variable(tf.random_normal([n_hidden_4])),
    'out':tf.Variable(tf.random_normal([n_classes]))
}

pred=multiplayer_perceptron(x,weights,biases)

# define loss and optimizer
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred,labels=y))
optimizer=tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# initilizing the variables
init=tf.global_variables_initializer()

In [39]:
# launch the graph
with tf.Session() as sess:
    sess.run(init)
    
    # Training 
    
    for epoch in range(training_epoch):
        avg_cost=0.
        total_batch=batch_size
        for i in range(total_batch):
            #im = np.reshape(image[0], (-1, n_input))
            batch_x=image
            batch_y=data.iloc[0:50,1]
            _,c=sess.run([optimizer,cost],feed_dict={x:batch_x,y:batch_y})
            avg_cost+=c/total_batch
        if epoch%display_step==0:
               print "Epoch:", '%04d' % (epoch+1), "cost=", \
                "{:.9f}".format(avg_cost)
    print "Optimization Finished!"
      # Test model
    correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    # Calculate accuracy
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print "Accuracy:", accuracy.eval({x: batch_x, y: batch_y})
        

ResourceExhaustedError: OOM when allocating tensor with shape[614400,256]
	 [[Node: random_normal_18/RandomStandardNormal = RandomStandardNormal[T=DT_INT32, dtype=DT_FLOAT, seed=0, seed2=0, _device="/job:localhost/replica:0/task:0/gpu:0"](random_normal_18/shape)]]

Caused by op u'random_normal_18/RandomStandardNormal', defined at:
  File "/home/edward/anaconda2/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/home/edward/anaconda2/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/home/edward/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/edward/anaconda2/lib/python2.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/edward/anaconda2/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 474, in start
    ioloop.IOLoop.instance().start()
  File "/home/edwar